## **Today we'll build a ScanningAgent that looks for promising deals by subscribing to RSS feeds**

In [29]:
# imports 
import os 
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [30]:
# Initializing the constants 

load_dotenv(override=True)

MODEL = "gpt-4o-mini"
openai = OpenAI()

In [31]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [02:45<00:00, 33.05s/it]


In [32]:
# as we have 10 deals from all 5 feeds each

len(deals)

50

In [33]:
deals[44].describe()

"Title: Buffalo Wild Wings Pick Six Offer From $20\nDetails: Buffalo Wild Wings' Pick Six offer has returned, which allows customers to get a meal for two, starting at $20. Price varies by region.If you're in the mood for fast food, you can save even more with these\xa0Uber Eats promo codes. Buy Now at Buffalo Wild Wings\nFeatures: \nURL: https://www.dealnews.com/Buffalo-Wild-Wings-Pick-Six-Offer-From-20/21763949.html?iref=rss-c196"

In [50]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response. 
Most important is that you respond with the 5 deals that have the most detailed product with price. It's not important to mention the terms of the deal; most important in thorough description of the product. 
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 


{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragraph of text for each item you choose.", 
        "price": 99.99, 
        "url": "the url as provided"
    },
    ,,,
]}
"""

In [51]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [52]:
# the user prompt is way too long, printing only 2000 chars

print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Newegg Shell Shocker Deals: Up to 42% off + free shipping
Details: Save on a large selection of computers, processors, graphics cards, motherboards, external and internal hard drives, SSDs, memory cards, upgrades, and accessories. Some have extra discount coupons on their product pages. We've pictured the Seagate Exos X20 20TB SATA 6.0Gbps 3.5"Internal Hard Drive fo

In [53]:
def get_recommendations():
    completion = openai.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt}, 
            {"role": "user", "content": user_prompt}
        ], 
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [54]:
result = get_recommendations()

In [55]:
len(result.deals)

5

In [56]:
print(result.deals[4])

product_description='The Samsung 990 EVO Plus 2TB Internal SSD delivers lightning-fast read and write speeds of up to 7,250 MB/s and 6,300 MB/s respectively, ensuring outstanding performance for gaming and high-demand applications. This storage solution serves as a perfect upgrade for those demanding speed and efficiency, suitable for both consumer and professional settings. Priced competitively at $120, it provides an excellent option for enhancing your system’s storage capabilities without compromising on performance.' price=120.0 url='https://www.dealnews.com/products/Samsung/Samsung-990-EVO-Plus-2-TB-Internal-SSD/486663.html?iref=rss-c39'


#### **Doing the same with our Production-ready ```ScannerAgent``` Class**

In [46]:
from agents.scanner_agent import ScannerAgent

In [47]:
agent = ScannerAgent()
result = agent.scan()

In [49]:
result

DealSelection(deals=[Deal(product_description="The Anker Solix C300X is a robust portable power station with a 288Wh capacity, ideal for outdoor adventures and emergency situations. With a 300W output and seven outlets including two 140W two-way USB-C fast charging ports, it ensures you can keep your devices powered wherever you go. This unit is bundled with a 60W solar panel for eco-friendly charging, allowing you to harness the sun's power for sustainable energy use. Its design is compact yet powerful, making it a perfect companion for camping, road trips, or home backup.", price=237.0, url='https://www.dealnews.com/products/Anker/Anker-Solix-C300-X-288-Wh-Portable-Power-Station-w-60-W-Solar-Panel/492331.html?iref=rss-c142'), Deal(product_description="Experience immersive audio with the Samsung Galaxy Buds3, designed for ultimate noise cancellation and comfort. These earbuds offer a sleek and compact design, making them perfect for on-the-go listening. They feature advanced sound tec